### Intended as a script which generates a .cvs report in one run

In [1]:
import tensorflow as tf;
import numpy as np;
import matplotlib.pyplot as plt;
print(tf.__version__);
import pandas as pd;

from sklearn.preprocessing import StandardScaler;
from sklearn.metrics import mean_squared_error;

2.3.0


In [2]:
pd.set_option('display.max_columns', 300);
pd.set_option('display.max_rows', 300);
tf.keras.backend.set_floatx('float64');

In [3]:
from tensorflow.python.client import device_lib

def get_available_gpus():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos if x.device_type == 'GPU']

get_available_gpus()

[]

In [4]:
MONTHS = 72;
SPLIT = 60; # 2014-2018: training, 2019: testing.
# BATCH_SIZE = 19; # used in NN_v1
BATCH_SIZE = 30;
WINDOW_SIZE = 1;

TEST_LENGTH = MONTHS - SPLIT;

### Data preparation

In [5]:
multi_data = pd.read_csv('../data/zri_acs_bikeshare_merged.csv', index_col = 0);
zip_ids = multi_data.index.unique();

multi_data.drop(["City", "State", "Metro", "CountyName", "year", "month", "datetime"],\
                 axis = 1, inplace = True);

multi_data.head()

,zri,percent_white,percent_black,percent_asian,percent_hispanic,percent_native_am,percent_other_race,percent_0_17,percent_18_39,percent_40_64,percent_65+,percent_rental_units_vacant,percent_rental_units_occupied,percent_graduate_deg,percent_bachelors,percent_associates,percent_highschool,percent_less_highschool,percent_commute_public_transport,percent_commute_less_30,percent_buildings_less_10_units,percent_buildings_10_19_units,percent_buildings_20_49_units,percent_buildings_50+_units,percent_commute_30_to_59,percent_commute_60_to_89,percent_commute_90_more,percent_new_city,percent_new_unit,percent_units_owner_occupied,median_building_age,income_per_capita,poverty_rate,total_pop,percent_workforce_unemployed,percent_work_from_home,median_age,percent_female,gini_index,percent_not_us_citizen,bs_total_stations,bs_total_systems,has_bike_sharing
zip,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1013,928.0,0.728315,0.020951,0.013732,0.219830,0.0,0.004959,0.230010,0.311464,0.309663,0.148864,0.068131,0.931869,0.028046,0.056892,0.259500,0.384872,0.225671,0.023499,0.741732,0.890613,0.030177,0.034856,0.044355,0.227721,0.017344,0.013031,0.108543,0.040940,0.443651,62.0,20475.25,0.191440,22343.583333,0.104130,0.009346,36.416667,0.509533,0.420742,0.056062,0.0,0.0,0
1013,931.0,0.727052,0.021179,0.013708,0.220867,0.0,0.004917,0.229526,0.311552,0.310406,0.148516,0.067207,0.932793,0.028179,0.057377,0.259450,0.384586,0.225073,0.023365,0.740587,0.889943,0.030154,0.035083,0.044820,0.227757,0.017701,0.013401,0.108828,0.040926,0.441963,62.0,20517.50,0.192359,22296.166667,0.104147,0.009839,36.433333,0.508816,0.420483,0.056165,0.0,0.0,0
1013,934.0,0.725788,0.021408,0.013685,0.221904,0.0,0.004874,0.229041,0.311641,0.311150,0.148168,0.066283,0.933717,0.028313,0.057863,0.259400,0.384301,0.224475,0.023230,0.739441,0.889273,0.030131,0.035311,0.045285,0.227792,0.018058,0.013771,0.109112,0.040912,0.440275,62.0,20559.75,0.193277,22248.750000,0.104165,0.010332,36.450000,0.508100,0.420225,0.056267,0.0,0.0,0
1013,929.0,0.724525,0.021636,0.013661,0.222940,0.0,0.004832,0.228557,0.311729,0.311894,0.147820,0.065359,0.934641,0.028446,0.058348,0.259350,0.384016,0.223877,0.023096,0.738295,0.888603,0.030108,0.035538,0.045750,0.227828,0.018415,0.014142,0.109397,0.040897,0.438587,62.0,20602.00,0.194196,22201.333333,0.104182,0.010825,36.466667,0.507383,0.419967,0.056370,0.0,0.0,0
1013,929.0,0.723261,0.021864,0.013638,0.223977,0.0,0.004789,0.228072,0.311818,0.312638,0.147472,0.064435,0.935565,0.028579,0.058833,0.259299,0.383730,0.223279,0.022961,0.737149,0.887934,0.030086,0.035766,0.046215,0.227864,0.018772,0.014512,0.109682,0.040883,0.436898,62.0,20644.25,0.195114,22153.916667,0.104199,0.011318,36.483333,0.506666,0.419708,0.056472,0.0,0.0,0


In [6]:
FEATURES = multi_data.shape[1] - 1;

### Utility functions

In [7]:
@tf.autograph.experimental.do_not_convert
def windowed_dataset(series, window_size, batch_size, shuffle_buffer):
    dataset = tf.data.Dataset.from_tensor_slices(series); #(43,)
    dataset = dataset.window(window_size + 1, shift=1, drop_remainder=True);
    dataset = dataset.flat_map(lambda window: window.batch(window_size + 1)); #(13,43)
    dataset = dataset.shuffle(shuffle_buffer)\
                     .map(lambda window: (window[:-1], window[-1][0]));
    dataset = dataset.batch(batch_size).prefetch(1);
    return dataset;

In [8]:
def plot_series(time, series, format="-", start=0, end=None):
    plt.plot(time[start:end], series[start:end], format)
    plt.xlabel("Time Frame")
    plt.ylabel("ZRI")
    plt.grid(True)

### Neural network center

In [9]:
class myCallbacks(tf.keras.callbacks.Callback):
    def on_epoch_end(self, epoch, logs={}):
        mse = logs.get("mse");
        if(mse < 0.0025):
            print("\nGot an mse at {:.4f} in round {} and stopped training\n".format(mse, epoch));
            self.model.stop_training = True;

In [10]:
def NN_model(dataset, test = None):
    tf.keras.backend.clear_session();
    # dataset = windowed_dataset(x_train, window_size, batch_size, shuffle_buffer_size)

    callback = myCallbacks();
    
    model = tf.keras.models.Sequential([
#         tf.keras.layers.Lambda(lambda x: tf.expand_dims(x, axis=-1),
#                           input_shape=[None]),
#       tf.keras.layers.Conv1D(filters=32, kernel_size=3,
#                           strides=1, padding="causal",
#                           activation="relu",
#                           input_shape=[None, WINDOW_SIZE, FEATURES+1]),
        tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True, input_shape = [None, WINDOW_SIZE, FEATURES+1])),
        #tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(16, return_sequences=True)),
        # tf.keras.layers.SimpleRNN(8, return_sequences=True),
        #  tf.keras.layers.SimpleRNN(16, return_sequences=True),
        tf.keras.layers.Dense(8, activation="relu"),
    #   tf.keras.layers.Dense(16, activation="relu"),
        tf.keras.layers.Dense(1)
        # tf.keras.layers.Lambda(lambda x: x * 2.0)
    ]);
    
    optimizer = tf.keras.optimizers.SGD(learning_rate=3e-4, momentum=0.9)
    model.compile(loss=tf.keras.losses.Huber(),
                  optimizer="adam",
                  metrics=["mae", "mse"])

    model.build((None,12,43))
    # model.summary()

    if not test: history = model.fit(dataset, epochs=500, callbacks = [callback], verbose = 0);
    else: history = model.fit(dataset, epochs=500, validation_data=test,\
                              callbacks=[callback], verbose = 0);
    return model;

In [11]:
def NN_forecast(model, series_transformed):
    forecast = []
    results = []
    for time in range(MONTHS - WINDOW_SIZE):
        forecast.append(model.predict(series_transformed[np.newaxis, time:time + WINDOW_SIZE, :]))

    results = [float(x[-1][0]) for x in forecast];
    timeline = range(WINDOW_SIZE, MONTHS);
    time_test = range(SPLIT, MONTHS);
    actual = list(series_transformed[WINDOW_SIZE:, 0]);

    forecast = series_transformed[SPLIT - WINDOW_SIZE:,:].copy();

    for time in range(TEST_LENGTH): # Change temp[time + WINDOW_SIZE]
        forecast[time + WINDOW_SIZE, 0] =\
        model.predict(forecast[np.newaxis, time:time + WINDOW_SIZE, :])[-1][0];

    pure_forecast = forecast[WINDOW_SIZE:,0];
    
    print(len(results), len(actual), len(pure_forecast))
    
    return results, actual, pure_forecast;

In [12]:
@tf.autograph.experimental.do_not_convert
def NN_test(ZONE, plot=False):
    '''
    Input: ZONE
    Output: the RMSE of a NN model on the predicted train, partially predicted test, and complete predicted test.
    '''
    # Collection of data
    series_frame = multi_data[multi_data.index == ZONE];
    # series_frame.sort_values("datetime", ascending = True, inplace = True);
    
    # Standardization
    scaler = StandardScaler();
    series_transformed = scaler.fit_transform(series_frame);
    
    # Train test split
    series_train = series_transformed[:SPLIT];
    series_test = series_transformed[SPLIT-WINDOW_SIZE:];
    
    # Window the training set to make input of the NN
    dataset_train = windowed_dataset(series_train, WINDOW_SIZE, BATCH_SIZE, 60);
    # dataset_test = windowed_dataset(series_test, WINDOW_SIZE, BATCH_SIZE, 60);
    
    model = NN_model(dataset_train);
    
    time_train = list(range(SPLIT));
    time_test = list(range(SPLIT, MONTHS));
    
    # Forecasting
    results, actual, pure_forecast = NN_forecast(model, series_transformed);
    
    # Compute MSE
    MSE_train = mean_squared_error(actual[:-TEST_LENGTH], results[:-TEST_LENGTH])**0.5 * scaler.var_[0]**0.5;
    MSE_test = mean_squared_error(actual[-TEST_LENGTH:], results[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    MSE_pure = mean_squared_error(actual[-TEST_LENGTH:], pure_forecast[-TEST_LENGTH:])**0.5 * scaler.var_[0]**0.5;
    
    if plot: # If the plot option is selected, plot the graph.
        time_actual = range(WINDOW_SIZE, MONTHS);
        plt.figure(figsize=(10, 6))
        plot_series(time_actual, actual);
        plot_series(time_actual, results);
        plot_series(time_test, pure_forecast);
        plt.show();

    return MSE_train, MSE_test, MSE_pure,\
        np.array(pure_forecast[-TEST_LENGTH:]) * scaler.var_[0]**0.5 + scaler.mean_[0];

### The script

In [13]:
score_dict = {"zip":[], "RMSE_train":[], "RMSE_test":[], "RMSE_pure":[], "forecast":[]};
with tf.device('/cpu:0'):
    for ZONE in zip_ids:
        %time M_train, M_test, M_pure, forecast = NN_test(ZONE, plot=False);
        print(ZONE, M_train, M_test, M_pure, forecast);

        score_dict["zip"].append(ZONE);
        score_dict["RMSE_train"].append(M_train);
        score_dict["RMSE_test"].append(M_test);
        score_dict["RMSE_pure"].append(M_pure);
        score_dict["forecast"].append(forecast);


Got an mse at 0.0025 in round 161 and stopped training

71 71 12
Wall time: 8.02 s
1013 3.975702876120819 71.18228774192413 70.67610987464383 [1100.19126078 1095.44701796 1090.32668593 1085.01439083 1079.94313951
 1075.15389544 1069.94608124 1065.76884541 1062.24291604 1059.03748831
 1056.12171823 1053.45762646]

Got an mse at 0.0025 in round 125 and stopped training

71 71 12
Wall time: 7 s
1020 4.0517844598098405 41.79868466794937 43.31305597892657 [1124.78580284 1125.70406584 1126.61525727 1129.40430581 1132.13835737
 1134.53397516 1135.10038415 1135.37801639 1135.50959214 1135.48741576
 1135.31107353 1134.99939838]
71 71 12
Wall time: 13.7 s
1040 3.242014990646429 17.477716519067336 17.926562206169226 [1097.36978758 1103.27220032 1107.88147035 1112.07464382 1116.17374807
 1119.84772166 1122.95323972 1125.53879707 1127.76543561 1129.79770876
 1131.76323373 1133.75946479]

Got an mse at 0.0025 in round 265 and stopped training

71 71 12
Wall time: 9.88 s
1085 3.865210966976414 51.96

71 71 12
Wall time: 11.4 s
1845 4.719600632623277 60.41367474579453 60.38208632545532 [1814.23546916 1814.39648831 1814.1674126  1813.14041566 1811.3467772
 1808.91890351 1806.03368272 1802.87349893 1796.65148747 1789.61765589
 1782.53656913 1776.44308907]

Got an mse at 0.0025 in round 351 and stopped training

71 71 12
Wall time: 13.1 s
1850 6.826003166817179 95.75501076598445 91.03379947958852 [2000.45951453 1999.61026143 1998.65403325 1995.72364874 1992.80305009
 1989.79729823 1986.72809842 1983.59822113 1980.39463686 1977.08941189
 1973.64213634 1970.00293332]

Got an mse at 0.0025 in round 389 and stopped training

71 71 12
Wall time: 13.9 s
1851 6.125960118520111 99.51688514403818 99.18192544171097 [1994.22866959 1991.02919163 1987.38531527 1983.09009153 1977.97082191
 1971.99459237 1965.44890383 1959.81870039 1952.80584737 1944.89764722
 1937.09008866 1928.34961568]

Got an mse at 0.0025 in round 266 and stopped training

71 71 12
Wall time: 11.5 s
1852 5.979525303579565 120.92

71 71 12
Wall time: 17 s
2127 7.200380113808559 160.29262222618297 172.27656153091266 [2871.91845037 2857.79790667 2850.96491557 2844.84568265 2838.58494678
 2830.35886492 2821.23129364 2811.63671931 2801.6601781  2791.38317627
 2781.61228266 2773.73224254]

Got an mse at 0.0025 in round 215 and stopped training

71 71 12
Wall time: 10.7 s
2128 7.875033757105688 222.26976279237695 209.62311234262984 [2472.2874758  2463.43111855 2457.86333762 2451.65138839 2445.09955621
 2438.38580742 2431.98514864 2425.84204156 2420.01848873 2411.95090412
 2402.85926275 2394.56991421]
71 71 12
Wall time: 16.5 s
2129 9.305276676752097 150.58219037015914 150.27989028316279 [2785.63247589 2776.36412234 2771.85755013 2768.72363953 2766.70913337
 2765.3975994  2764.10526314 2762.87351109 2761.53326109 2760.06548152
 2758.52748231 2756.98501728]
71 71 12
Wall time: 16.9 s
2130 7.035422889933633 218.9646870474748 218.58702766429644 [2600.39736658 2600.49754461 2601.63826103 2602.90937484 2604.26534392
 2605.6

71 71 12
Wall time: 11.2 s
2215 7.688490059844994 128.6249161543384 176.95947536827504 [2353.37061249 2352.94363119 2357.93440393 2364.39921769 2371.70661997
 2379.67866015 2387.89028496 2394.37106797 2400.02142384 2404.83847949
 2408.58290422 2410.2846343 ]

Got an mse at 0.0025 in round 222 and stopped training

71 71 12
Wall time: 10.8 s
2301 7.1131145406019884 100.4042588456105 96.89714751272771 [1650.73445904 1640.91554878 1633.2694328  1625.63580798 1618.65164458
 1612.22210386 1606.23049412 1600.67061778 1595.55935898 1590.88779198
 1586.60813712 1582.6425735 ]

Got an mse at 0.0025 in round 108 and stopped training

71 71 12
Wall time: 8.16 s
2302 7.128601194920557 107.71686759685468 114.7462541867044 [1742.64024646 1735.40430839 1726.38506118 1716.05236457 1701.94240261
 1686.57889762 1666.22190692 1646.54063008 1627.86546371 1610.64361583
 1595.66729847 1583.38652127]

Got an mse at 0.0025 in round 163 and stopped training

71 71 12
Wall time: 9.4 s
2360 7.851500938711904 133

71 71 12
Wall time: 12.4 s
2895 4.0245156813244725 73.29897716525917 73.15218961060137 [1082.49757441 1078.68184423 1074.77848484 1071.18847199 1068.4468376
 1065.68338038 1062.80402712 1059.70963618 1055.51085163 1050.17867039
 1044.25106268 1037.41250029]

Got an mse at 0.0025 in round 293 and stopped training

71 71 12
Wall time: 12.5 s
2904 4.527246810971782 45.82945486912926 48.46187328784997 [1256.95299664 1253.67785834 1248.4514297  1242.25900057 1233.40788608
 1222.24677597 1208.50350881 1192.25021114 1175.01054524 1159.32724101
 1146.61671537 1136.8150967 ]
71 71 12
Wall time: 16.8 s
2905 38.61913443482064 78.42483643531261 87.89799068883198 [1273.66423041 1267.18473176 1258.49357514 1247.42852977 1233.95940761
 1220.28224749 1207.23637521 1194.41047991 1183.8918081  1176.82078969
 1171.01541354 1167.26337883]

Got an mse at 0.0025 in round 73 and stopped training

71 71 12
Wall time: 7.74 s
2906 5.544229386555205 78.84144768323658 81.09134172818534 [1569.69765918 1560.8538067

...


Got an mse at 0.0025 in round 63 and stopped training

71 71 12
Wall time: 6.04 s
92805 10.253561457481672 106.98232027582797 111.23101925041864 [1953.99685861 1955.55700385 1956.8605926  1957.40054151 1956.12297345
 1953.69441134 1950.23751869 1946.06215427 1941.67768337 1937.61673116
 1934.21254084 1931.52816641]

Got an mse at 0.0025 in round 90 and stopped training

71 71 12
Wall time: 6.59 s
92806 10.702072589942128 219.7149708237921 225.5933198332737 [2183.68899361 2176.85803566 2168.1322815  2158.06067133 2146.94980366
 2135.00317489 2122.57534519 2109.98244956 2096.6405049  2083.82514434
 2071.70802678 2060.25626684]

Got an mse at 0.0025 in round 107 and stopped training

71 71 12
Wall time: 7 s
92832 6.802620892336601 122.03961894963827 127.25052429433583 [1951.86236143 1944.67205739 1933.37596574 1920.6421913  1900.33324737
 1881.11736533 1871.86734769 1865.62415087 1861.37802439 1858.59733298
 1856.87395882 1855.9091016 ]

Got an mse at 0.0025 in round 188 and stopped trai


Got an mse at 0.0025 in round 125 and stopped training

71 71 12
Wall time: 7.25 s
94401 11.593935073726456 56.61921481655412 58.50604456017354 [2839.13532749 2808.62435008 2816.09607344 2825.72300873 2834.78828874
 2843.33885625 2851.60784155 2859.741843   2862.21793602 2863.40147422
 2862.0124225  2860.26625917]

Got an mse at 0.0025 in round 76 and stopped training

71 71 12
Wall time: 6.54 s
94403 12.059830419726474 114.69094035928553 119.20785991088567 [3130.35644047 3067.74781616 3065.76403482 3064.54452661 3061.36487441
 3057.87237433 3054.24537483 3050.58204943 3046.96210482 3043.45028965
 3039.85947116 3036.04154527]

Got an mse at 0.0025 in round 114 and stopped training

71 71 12
Wall time: 7.53 s
94404 12.678509898656088 129.39107330064076 132.0499905497596 [3166.08655456 3164.12136391 3162.14806062 3160.20046592 3158.03859687
 3155.25939037 3151.24949775 3145.1734533  3136.16399404 3124.29374251
 3111.13584491 3099.17256029]

Got an mse at 0.0025 in round 214 and stopped 


Got an mse at 0.0025 in round 138 and stopped training

71 71 12
Wall time: 7.54 s
95128 7.146328239929348 149.66615996352814 146.61962620238168 [2542.97363535 2539.55681615 2535.82078141 2531.4083502  2524.46490647
 2517.04618053 2509.59864249 2502.38618282 2495.67817034 2490.19619649
 2485.8805474  2482.42747796]

Got an mse at 0.0025 in round 136 and stopped training

71 71 12
Wall time: 7.63 s
95134 10.358608916044307 110.91995799096368 114.77934847104362 [3165.62956356 3172.2702556  3176.43008449 3179.77770482 3182.49553757
 3184.68662074 3186.45978884 3187.9178631  3189.14433511 3190.19734737
 3191.1106986  3191.89918668]

Got an mse at 0.0025 in round 94 and stopped training

71 71 12
Wall time: 6.72 s
95207 9.151970509566295 142.14548551024174 146.4005727450076 [1238.12059217 1216.9012268  1195.31088933 1175.02289755 1155.98615016
 1137.65608918 1119.80608575 1102.59230743 1086.38837154 1075.22733465
 1068.45304427 1062.59653528]

Got an mse at 0.0025 in round 183 and stopped 

71 71 12
Wall time: 7.19 s
98012 7.423846358739626 63.768059986275205 65.09870572197792 [1822.79736075 1812.93721606 1798.23421418 1781.53523763 1764.86845105
 1749.44299984 1735.83722881 1724.41073201 1715.0970503  1707.44641614
 1698.63836816 1687.29151318]

Got an mse at 0.0025 in round 188 and stopped training

71 71 12
Wall time: 8.57 s
98029 8.25024670256617 70.30743060869018 74.63449566553543 [2126.04249554 2125.7166553  2123.25529762 2120.5145061  2117.61005175
 2114.45766185 2111.03693275 2107.47485094 2103.94444623 2100.49749769
 2097.0509771  2093.50482564]

Got an mse at 0.0025 in round 98 and stopped training

71 71 12
Wall time: 6.88 s
98034 7.043226408737311 28.862495374817236 28.732168404270315 [1845.88867712 1846.43978829 1846.88249398 1847.59809116 1848.35590568
 1847.5073684  1846.45290227 1845.12675163 1843.44581897 1841.3758501
 1837.25565626 1827.49464295]

Got an mse at 0.0025 in round 170 and stopped training

71 71 12
Wall time: 8.12 s
98052 7.922477481462148 5

71 71 12
Wall time: 14 s
99501 1.960926059727655 33.154867817983664 35.25148705965249 [1270.37256885 1270.14540243 1269.85547264 1269.54359286 1269.19320171
 1268.6103406  1267.09490973 1265.38727273 1263.58360713 1262.55154858
 1261.58520802 1260.92725486]
71 71 12
Wall time: 14.4 s
99504 2.941504108632114 17.056383253872127 18.591604270300415 [1433.53210991 1432.97532695 1433.02103812 1433.50520699 1434.38298947
 1435.61588713 1437.87912887 1440.64269171 1443.58469257 1446.49652865
 1449.24797549 1451.76574641]
71 71 12
Wall time: 14.8 s
99508 2.422134922875056 23.209348066313478 25.426267200083913 [1283.37357803 1287.11813171 1290.31718753 1293.01020233 1294.81737329
 1296.20957313 1297.38820301 1298.40770186 1299.30567599 1300.11048698
 1300.84327412 1301.51946458]
71 71 12
Wall time: 14.4 s
99654 4.526265768067546 34.59126037973857 33.93552707133203 [1338.85443466 1335.32998518 1331.70471475 1328.04437853 1324.43437133
 1320.9290789  1317.21689999 1313.59112634 1310.13942455 1306.

In [14]:
print(len(score_dict["RMSE_pure"]), sum(score_dict["RMSE_test"])/len(score_dict["RMSE_test"]),
      sum(score_dict["RMSE_pure"])/len(score_dict["RMSE_pure"]));

1301 61.974812829636846 63.80246371979302


In [15]:
test = pd.DataFrame(score_dict);
test.head()

,zip,RMSE_train,RMSE_test,RMSE_pure,forecast
0,1013,3.975703,71.182288,70.676110,"[1100.1912607799838, 1095.4470179598954, 1090...."
1,1020,4.051784,41.798685,43.313056,"[1124.7858028380701, 1125.704065841953, 1126.6..."
2,1040,3.242015,17.477717,17.926562,"[1097.3697875760759, 1103.2722003182564, 1107...."
3,1085,3.865211,51.968525,59.333081,"[1143.6823771171785, 1140.8718704679395, 1139...."
4,1104,3.970878,69.215055,74.081186,"[1117.8334233729324, 1084.8622792878048, 1080...."


In [16]:
test.to_csv('NN_feature_window_1_overoverfit.csv', index = False)